# COVID-19 Data Analysis
> Source: JHU . 

- comments: true
- author: Sonal Kelwadkar
- hide: false
- image: images/covid-permillion-trajectories.png
- permalink: /covid-compare-permillion/

In [1]:
#collapse-hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
"""
import os
import matplotlib
from IPython.display import display, Markdown
from collections import defaultdict
from datetime import datetime, timedelta, date
import scipy as sci
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pycountry_convert as pc
import folium
import branca
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests
from keras.layers import Input, Dense, Activation, LeakyReLU, Dropout
from keras import models
from keras.optimizers import RMSprop, Adam
from folium import plugins
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
import calmap"""

%config InlineBackend.figure_format = 'retina'

chart_width = 550
chart_height= 400

Using TensorFlow backend.


## Deaths Per Million Of Inhabitants

Since reaching at least 1 death per million

> Tip: Click (Shift+ for multiple) on countries in the legend to filter the visualization.

In [2]:
#collapse-hide
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

df_covid19 = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")
df_table = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_time.csv",parse_dates=['Last_Update'])


In [3]:
#df_pop = pd.read_csv('locations_population.csv')

In [4]:
#collapse-hide
df_covid19 = df_covid19.drop(["People_Tested","People_Hospitalized","UID","ISO3","Mortality_Rate"],axis =1)
df_confirmed = df_confirmed.rename(columns={"Province/State":"state","Country/Region": "country"})
df_deaths = df_deaths.rename(columns={"Province/State":"state","Country/Region": "country"})
df_recovered = df_recovered.rename(columns={"Province/State":"state","Country/Region": "country"})
df_covid19 = df_covid19.rename(columns={"Country_Region": "country"})
df_covid19["Active"] = df_covid19["Confirmed"]-df_covid19["Recovered"]-df_covid19["Deaths"]


# Changing the conuntry names as required by pycountry_convert Lib
df_confirmed.loc[df_confirmed['country'] == "US", "country"] = "USA"
df_deaths.loc[df_deaths['country'] == "US", "country"] = "USA"
df_covid19.loc[df_covid19['country'] == "US", "country"] = "USA"
df_table.loc[df_table['Country_Region'] == "US", "Country_Region"] = "USA"
df_recovered.loc[df_recovered['country'] == "US", "country"] = "USA"


df_confirmed.loc[df_confirmed['country'] == 'Korea, South', "country"] = 'South Korea'
df_deaths.loc[df_deaths['country'] == 'Korea, South', "country"] = 'South Korea'
df_covid19.loc[df_covid19['country'] == "Korea, South", "country"] = "South Korea"
df_table.loc[df_table['Country_Region'] == "Korea, South", "Country_Region"] = "South Korea"
df_recovered.loc[df_recovered['country'] == 'Korea, South', "country"] = 'South Korea'

df_confirmed.loc[df_confirmed['country'] == 'Taiwan*', "country"] = 'Taiwan'
df_deaths.loc[df_deaths['country'] == 'Taiwan*', "country"] = 'Taiwan'
df_covid19.loc[df_covid19['country'] == "Taiwan*", "country"] = "Taiwan"
df_table.loc[df_table['Country_Region'] == "Taiwan*", "Country_Region"] = "Taiwan"
df_recovered.loc[df_recovered['country'] == 'Taiwan*', "country"] = 'Taiwan'

df_confirmed.loc[df_confirmed['country'] == 'Congo (Kinshasa)', "country"] = 'Democratic Republic of the Congo'
df_deaths.loc[df_deaths['country'] == 'Congo (Kinshasa)', "country"] = 'Democratic Republic of the Congo'
df_covid19.loc[df_covid19['country'] == "Congo (Kinshasa)", "country"] = "Democratic Republic of the Congo"
df_table.loc[df_table['Country_Region'] == "Congo (Kinshasa)", "Country_Region"] = "Democratic Republic of the Congo"
df_recovered.loc[df_recovered['country'] == 'Congo (Kinshasa)', "country"] = 'Democratic Republic of the Congo'

df_confirmed.loc[df_confirmed['country'] == "Cote d'Ivoire", "country"] = "Côte d'Ivoire"
df_deaths.loc[df_deaths['country'] == "Cote d'Ivoire", "country"] = "Côte d'Ivoire"
df_covid19.loc[df_covid19['country'] == "Cote d'Ivoire", "country"] = "Côte d'Ivoire"
df_table.loc[df_table['Country_Region'] == "Cote d'Ivoire", "Country_Region"] = "Côte d'Ivoire"
df_recovered.loc[df_recovered['country'] == "Cote d'Ivoire", "country"] = "Côte d'Ivoire"

df_confirmed.loc[df_confirmed['country'] == "Reunion", "country"] = "Réunion"
df_deaths.loc[df_deaths['country'] == "Reunion", "country"] = "Réunion"
df_covid19.loc[df_covid19['country'] == "Reunion", "country"] = "Réunion"
df_table.loc[df_table['Country_Region'] == "Reunion", "Country_Region"] = "Réunion"
df_recovered.loc[df_recovered['country'] == "Reunion", "country"] = "Réunion"

df_confirmed.loc[df_confirmed['country'] == 'Congo (Brazzaville)', "country"] = 'Republic of the Congo'
df_deaths.loc[df_deaths['country'] == 'Congo (Brazzaville)', "country"] = 'Republic of the Congo'
df_covid19.loc[df_covid19['country'] == "Congo (Brazzaville)", "country"] = "Republic of the Congo"
df_table.loc[df_table['Country_Region'] == "Congo (Brazzaville)", "Country_Region"] = "Republic of the Congo"
df_recovered.loc[df_recovered['country'] == 'Congo (Brazzaville)', "country"] = 'Republic of the Congo'

df_confirmed.loc[df_confirmed['country'] == 'Bahamas, The', "country"] = 'Bahamas'
df_deaths.loc[df_deaths['country'] == 'Bahamas, The', "country"] = 'Bahamas'
df_covid19.loc[df_covid19['country'] == "Bahamas, The", "country"] = "Bahamas"
df_table.loc[df_table['Country_Region'] == "Bahamas, The", "Country_Region"] = "Bahamas"
df_recovered.loc[df_recovered['country'] == 'Bahamas, The', "country"] = 'Bahamas'

df_confirmed.loc[df_confirmed['country'] == 'Gambia, The', "country"] = 'Gambia'
df_deaths.loc[df_deaths['country'] == 'Gambia, The', "country"] = 'Gambia'
df_covid19.loc[df_covid19['country'] == "Gambia, The", "country"] = "Gambia"
df_table.loc[df_table['Country_Region'] == "Gambia", "Country_Region"] = "Gambia"
df_recovered.loc[df_recovered['country'] == 'Gambia, The', "country"] = 'Gambia'

In [5]:
'''df_pop = df_pop.rename(columns={"Country.Region":"country"})
df_pop.loc[df_pop['country'] == "US", "country"] = "USA"
df_pop.loc[df_pop['country'] == "Korea, South", "country"] = "South Korea"
df_pop.loc[df_pop['country'] == "Taiwan*", "country"] = "Taiwan"
df_pop.loc[df_pop['country'] == "Congo (Kinshasa)", "country"] = "Democratic Republic of the Congo"
df_pop.loc[df_pop['country'] == "Cote d'Ivoire", "country"] = "Côte d'Ivoire"
df_pop.loc[df_pop['country'] == "Reunion", "country"] = "Réunion"
df_pop.loc[df_pop['country'] == "Congo (Brazzaville)", "country"] = "Republic of the Congo"
df_pop.loc[df_pop['country'] == "Bahamas, The", "country"] = "Bahamas"
df_pop.loc[df_pop['country'] == "Gambia, The", "country"] = "Gambia"
'''

In [6]:
#collapse-hide
# getting all countries
countries = np.asarray(df_confirmed["country"])
countries1 = np.asarray(df_covid19["country"])
# Continent_code to Continent_names
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU' : 'Europe',
    'na' : 'Others'
}

# Defininng Function for getting continent code for country.
def country_to_continent_code(country):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))
    except :
        return 'na'

#Collecting Continent Information
df_confirmed.insert(2,"continent", [continents[country_to_continent_code(country)] for country in countries[:]])
df_deaths.insert(2,"continent",  [continents[country_to_continent_code(country)] for country in countries[:]])
df_covid19.insert(1,"continent",  [continents[country_to_continent_code(country)] for country in countries1[:]])
df_table.insert(1,"continent",  [continents[country_to_continent_code(country)] for country in df_table["Country_Region"].values])

In [7]:
#collapse-hide
df_table = df_table[df_table["continent"] != "Others"]
df_confirmed = df_confirmed.replace(np.nan, '', regex=True)
df_deaths = df_deaths.replace(np.nan, '', regex=True)

In [8]:
#collapse-hide
df_countries_cases = df_covid19.copy().drop(['Lat','Long_','continent','Last_Update'],axis =1)
df_countries_cases.index = df_countries_cases["country"]
df_countries_cases = df_countries_cases.drop(['country'],axis=1)

df_continents_cases = df_covid19.copy().drop(['Lat','Long_','country','Last_Update'],axis =1)
df_continents_cases = df_continents_cases.groupby(["continent"]).sum()

df_countries_cases.fillna(0,inplace=True)
df_continents_cases.fillna(0,inplace=True)

In [9]:
#collapse-hide
df_t = pd.DataFrame(pd.to_numeric(df_countries_cases.sum()),dtype=np.float64).transpose()
df_t["Mortality Rate (per 100)"] = np.round(100*df_t["Deaths"]/df_t["Confirmed"],2)
df_t.style.background_gradient(cmap='Wistia',axis=1).format("{:.2f}").format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"][0])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
0,20284882,741126.00,12585473.00,6853075.00,61458.14,3.65


In [10]:
#collapse-hide
df_continents_cases["Mortality Rate (per 100)"] = np.round(100*df_continents_cases["Deaths"]/df_continents_cases["Confirmed"],2)
df_continents_cases.style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='Pastel1_r',subset=["Incident_Rate"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
continent,,,,,,
Africa,1061880,23804,756425,281651,5255.89,2.24
Asia,5195243,112372,3948522,1134349,19652.53,2.16
Australia,23938,378,14010,9550,124.87,1.58
Europe,3060860,206143,1762892,986624,17634.63,6.73
North America,6083569,235551,2399857,3448161,7206.44,3.87
Others,41915,1059,25215,15634,3228.54,2.53
South America,4817477,161819,3678552,977106,8355.25,3.36


In [11]:
#collapse-hide
df_countries_cases["Mortality Rate (per 100)"] = np.round(100*df_countries_cases["Deaths"]/df_countries_cases["Confirmed"],2)
df_countries_cases.sort_values('Confirmed', ascending= False).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='Pastel1_r',subset=["Incident_Rate"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["Confirmed","Deaths","Recovered","Active"])

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality Rate (per 100)
country,,,,,,
USA,5141208,164537,1714960,3261711,1560.47,3.20
Brazil,3057470,101752,2449338,506380,1438.41,3.33
India,2329638,46091,1639599,643948,168.81,1.98
Russia,895691,15103,701796,178792,613.76,1.69
South Africa,566109,10751,426125,129233,954.51,1.90
Mexico,492522,53929,397278,41315,385.41,10.95
Peru,483133,21276,329404,132453,1465.29,4.40
Colombia,410453,13475,230427,166551,806.66,3.28
Chile,376616,10178,349541,16897,1970.14,2.70


In [12]:
#collapse-hide 
df_deaths
df_deaths.head()

,state,country,continent,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,,Afghanistan,Asia,33.93911,67.709953,0,0,0,0,0,...,1284,1288,1288,1294,1298,1307,1312,1312,1328,1344
1,,Albania,Europe,41.15330,20.168300,0,0,0,0,0,...,166,172,176,182,188,189,193,199,200,205
2,,Algeria,Africa,28.03390,1.659600,0,0,0,0,0,...,1231,1239,1248,1261,1273,1282,1293,1302,1312,1322
3,,Andorra,Europe,42.50630,1.521800,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52
4,,Angola,Africa,-11.20270,17.873900,0,0,0,0,0,...,55,58,59,62,64,67,70,75,78,80


In [13]:
#collapse-hide
#data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", 
#                   error_bad_lines=False)
data = df_deaths.copy()
data = data.drop(columns=["Lat", "Long"])
data = data.melt(id_vars= ["state", "country", "continent"])
data = pd.DataFrame(data.groupby(['country', "variable"]).sum())
data.reset_index(inplace=True)  
data = data.rename(columns={"country": "location", "variable": "date", "value": "total_cases"})
data['date'] = pd.to_datetime(data.date)
data = data.sort_values(by = "date")
data.loc[data.location == "US","location"] = "United States"
data.loc[data.location == "Korea, South","location"] = "South Korea"
data.head()

,location,date,total_cases
0,Afghanistan,2020-01-22,0
36946,Vietnam,2020-01-22,0
29232,San Marino,2020-01-22,0
1421,Armenia,2020-01-22,0
20300,Lithuania,2020-01-22,0


In [14]:
#collapse-hide 
data_pwt = pd.read_stata("https://www.rug.nl/ggdc/docs/pwt91.dta")

filter1 = data_pwt["year"] == 2017
data_pop = data_pwt[filter1]
data_pop = data_pop[["country","pop"]]
data_pop.loc[data_pop.country == "Republic of Korea","country"] = "South Korea"
data_pop.loc[data_pop.country == "Iran (Islamic Republic of)","country"] = "Iran"

# per habitant
data_pc = data.copy()
countries = ["Italy", "Spain", "France", "United Kingdom", "Germany", 
             "Portugal", "United States", "Singapore", "South Korea", "Japan", 
             "Brazil", "Iran", 'Netherlands', 'Belgium', 'Sweden', "Romania",
             'Switzerland', 'Norway', 'Denmark', 'Austria', 'Slovenia', 'Greece']
data_countries = []
data_countries_pc = []

# compute per habitant
for i in countries:
    data_pc.loc[data_pc.location == i,"total_cases"] = data_pc.loc[data_pc.location == i,"total_cases"]/float(data_pop.loc[data_pop.country == i, "pop"])

    # get each country time series
filter1 = data_pc["total_cases"] > 1
for i in countries:
    filter_country = data_pc["location"]== i
    data_countries_pc.append(data_pc[filter_country & filter1])

In [15]:
#collapse-hide
# Stack data to get it to Altair dataframe format
data_countries_pc2 = data_countries_pc.copy()
for i in range(0,len(countries)):
    data_countries_pc2[i] = data_countries_pc2[i].reset_index()
    data_countries_pc2[i]['n_days'] = data_countries_pc2[i].index
    data_countries_pc2[i]['log_cases'] = np.log(data_countries_pc2[i]["total_cases"])
data_plot = data_countries_pc2[0]
for i in range(1, len(countries)):    
    data_plot = pd.concat([data_plot, data_countries_pc2[i]], axis=0)
data_plot["trend_2days"] = np.log(2)/2*data_plot["n_days"]
data_plot["trend_4days"] = np.log(2)/4*data_plot["n_days"]
data_plot["trend_12days"] = np.log(2)/12*data_plot["n_days"]
data_plot["trend_2days_label"] = "Doubles every 2 days"
data_plot["trend_4days_label"] = "Doubles evey 4 days"
data_plot["trend_12days_label"] = "Doubles every 12 days"


# Plot it using Altair
source = data_plot

scales = alt.selection_interval(bind='scales', zoom=False)
selection = alt.selection_multi(fields=['location'], bind='legend')

base = alt.Chart(source, title = "COVID-19 Deaths Per Million of Inhabitants").encode(
    x = alt.X('n_days:Q', title = "Days passed since reaching 1 death per million"),
    y = alt.Y("log_cases:Q",title = "Log of deaths per million"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country", labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

trend_2d = alt.Chart(source).encode(
    x = "n_days:Q",
    y = alt.Y("trend_2days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
).mark_line(color="grey", strokeDash=[3,3])


labels = pd.DataFrame([{'label': 'Doubles every 2 days', 'x_coord': 11.4, 'y_coord': 6.5},
                       {'label': 'Doubles every 4 days', 'x_coord': 30.5, 'y_coord': 6.5},
                       {'label': 'Doubles every 12 days', 'x_coord': 46, 'y_coord': 3},
                      ])
trend_label = (alt.Chart(labels)
                    .mark_text(align='left', dx=-55, dy=-15, fontSize=12, color="grey")
                    .encode(x='x_coord:Q',
                            y='y_coord:Q',
                            text='label:N')
                   )

trend_4d = alt.Chart(source).mark_line(color="grey", strokeDash=[3,3]).encode(
    x = "n_days:Q",
    y = alt.Y("trend_4days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
)

trend_12d = alt.Chart(source).mark_line(color="grey", strokeDash=[3,3]).encode(
    x = "n_days:Q",
    y = alt.Y("trend_12days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
)


plot1= (
(trend_2d + trend_4d + trend_12d + trend_label + lines)
.configure_title(fontSize=20)
.configure_axis(labelFontSize=15,titleFontSize=18)
)
#plot1.save(("../images/covid-permillion-trajectories.png"))
plot1

alt.LayerChart(...)

Last Available Total Deaths By Country:

In [16]:
#collapse-hide
label = 'Deaths'
temp = pd.concat([x.copy() for x in data_countries_pc]).loc[lambda x: x.date >= '3/1/2020']

metric_name = f'{label} per Million'
temp.columns = ['Country', 'date', metric_name]
# temp.loc[:, 'month'] = temp.date.dt.strftime('%Y-%m')
temp.loc[:, f'Log of {label} per Million'] = temp[f'{label} per Million'].apply(lambda x: np.log(x))

temp.groupby('Country').last()

,date,Deaths per Million,Log of Deaths per Million
Country,,,
Austria,2020-08-11,82.766175,4.416019
Belgium,2020-08-11,864.879670,6.762590
Brazil,2020-08-11,486.181060,6.186581
Denmark,2020-08-11,108.309841,4.684996
France,2020-08-11,451.132636,6.111761
Germany,2020-08-11,112.136475,4.719717
Greece,2020-08-11,19.176018,2.953660
Iran,2020-08-11,231.633244,5.445155
Italy,2020-08-11,593.245594,6.385608


In [17]:
#collapse-hide
# Get data and clean it

#data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", 
#                   error_bad_lines=False)
data = df_confirmed.copy()
data = data.drop(columns=["Lat", "Long"])
data = data.melt(id_vars= ["state", "country", "continent"])
data = pd.DataFrame(data.groupby(['country', "variable"]).sum())
data.reset_index(inplace=True)  
data = data.rename(columns={"country": "location", "variable": "date", "value": "total_cases"})
data['date'] = pd.to_datetime(data.date)
data = data.sort_values(by = "date")
data.loc[data.location == "US","location"] = "United States"
data.loc[data.location == "Korea, South","location"] = "South Korea"

# Population data (last year is 2017 which is what we use)
data_pwt = pd.read_stata("https://www.rug.nl/ggdc/docs/pwt91.dta")

filter1 = data_pwt["year"] == 2017
data_pop = data_pwt[filter1]
data_pop = data_pop[["country","pop"]]
data_pop.loc[data_pop.country == "Republic of Korea","country"] = "South Korea"
data_pop.loc[data_pop.country == "Iran (Islamic Republic of)","country"] = "Iran"

# per habitant
data_pc = data.copy()

# I can add more countries if needed
countries = ["China", "Italy", "Spain", "France", "United Kingdom", "Germany", 
             "Portugal", "United States", "Singapore","South Korea", "Japan", 
             "Brazil", "Iran", "Romania"]

data_countries = []
data_countries_pc = []

# compute per habitant
for i in countries:
    data_pc.loc[data_pc.location == i,"total_cases"] = data_pc.loc[data_pc.location == i,"total_cases"]/float(data_pop.loc[data_pop.country == i, "pop"])
    
# get each country time series
filter1 = data_pc["total_cases"] > 1

for i in countries:
    filter_country = data_pc["location"]== i
    data_countries_pc.append(data_pc[filter_country & filter1])

## Cases Per Million of Habitants

Since reaching at least 1 case per million

> Note: The following chart, "Cases Per Million of Habitants" is biased depending on how widely a country administers tests. Please read with caution.

> Tip: Click (Shift+ for multiple) on countries in the legend to filter the visualization.

In [18]:
#collapse-hide
# Stack data to get it to Altair dataframe format
data_countries_pc2 = data_countries_pc.copy()
for i in range(0,len(countries)):
    data_countries_pc2[i] = data_countries_pc2[i].reset_index()
    data_countries_pc2[i]['n_days'] = data_countries_pc2[i].index
    data_countries_pc2[i]['log_cases'] = np.log(data_countries_pc2[i]["total_cases"])
data_plot = data_countries_pc2[0]
for i in range(1, len(countries)):    
    data_plot = pd.concat([data_plot, data_countries_pc2[i]], axis=0)
data_plot["trend_2days"] = np.log(2)/2*data_plot["n_days"]
data_plot["trend_4days"] = np.log(2)/4*data_plot["n_days"]
data_plot["trend_12days"] = np.log(2)/12*data_plot["n_days"]
data_plot["trend_2days_label"] = "Doubles every 2 days"
data_plot["trend_4days_label"] = "Doubles evey 4 days"
data_plot["trend_12days_label"] = "Doubles every 12 days"


# Plot it using Altair
source = data_plot

scales = alt.selection_interval(bind='scales', zoom=False)
selection = alt.selection_multi(fields=['location'], bind='legend')

base = alt.Chart(source, title = "COVID-19 Confirmed Cases Per Million of Inhabitants").encode(
    x = alt.X('n_days:Q', title = "Days passed since reaching 1 case per million"),
    y = alt.Y("log_cases:Q",title = "Log of confirmed cases per million"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country", labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
)

trend_2d = alt.Chart(source).encode(
    x = "n_days:Q",
    y = alt.Y("trend_2days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
).mark_line( strokeDash=[3,3], color="grey")

labels = pd.DataFrame([{'label': 'Doubles every 2 days', 'x_coord': 13, 'y_coord': 8.3},
                       {'label': 'Doubles every 4 days', 'x_coord': 37.5, 'y_coord': 8.3},
                       {'label': 'Doubles every 12 days', 'x_coord': 71, 'y_coord': 3},
                      ])
trend_label = (alt.Chart(labels)
                    .mark_text(align='left', dx=-55, dy=-15, fontSize=12, color="grey")
                    .encode(x='x_coord:Q',
                            y='y_coord:Q',
                            text='label:N')
                   )


trend_4d = alt.Chart(source).mark_line(color="grey", strokeDash=[3,3]).encode(
    x = "n_days:Q",
    y = alt.Y("trend_4days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
)

trend_12d = alt.Chart(source).mark_line(color="grey", strokeDash=[3,3]).encode(
    x = "n_days:Q",
    y = alt.Y("trend_12days:Q",  scale=alt.Scale(domain=(0, max(data_plot["log_cases"])))),
)

(
(trend_2d  + trend_4d + trend_12d + trend_label + lines)
.configure_title(fontSize=20)
.configure_axis(labelFontSize=15,titleFontSize=18)
)

alt.LayerChart(...)

In [19]:
#collapse-hide
df_confirmed.head()

,state,country,continent,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,,Afghanistan,Asia,33.93911,67.709953,0,0,0,0,0,...,36710,36747,36782,36829,36896,37015,37054,37054,37162,37269
1,,Albania,Europe,41.15330,20.168300,0,0,0,0,0,...,5519,5620,5750,5889,6016,6151,6275,6411,6536,6676
2,,Algeria,Africa,28.03390,1.659600,0,0,0,0,0,...,31465,31972,32504,33055,33626,34155,34693,35160,35712,36204
3,,Andorra,Europe,42.50630,1.521800,0,0,0,0,0,...,925,937,939,939,944,955,955,955,963,963
4,,Angola,Africa,-11.20270,17.873900,0,0,0,0,0,...,1199,1280,1344,1395,1483,1538,1572,1672,1679,1735


In [20]:
#collapse-hide
def load_data(name):
    base_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series'
    url = f'{base_url}/time_series_covid19_{name}_global.csv'
    df = pd.read_csv(url, 
                     index_col=['Country/Region', 'Province/State', 'Lat', 'Long'])
    
    df['type'] = name.lower()
    df.columns.name = 'date'

    df = (df.set_index('type', append=True)
            .reset_index(['Lat', 'Long'], drop=True)
            .stack()
            .reset_index()
            .set_index('date')
         )
    df.index = pd.to_datetime(df.index)
    df.columns = ['country', 'state', 'type', 'cases']

    # Move HK to country level
    df.loc[df.state =='Hong Kong', 'country'] = 'Hong Kong'
    df.loc[df.state =='Hong Kong', 'state'] = np.nan

    # Aggregate large countries split by states
    df = pd.concat([df, 
                    (df.loc[~df.state.isna()]
                     .groupby(['country', 'date', 'type'])
                     .sum()
                     .rename(index=lambda x: x+' (total)', level=0)
                     .reset_index(level=['country', 'type']))
                   ])
    return df


In [21]:
#collapse-hide

df_confirmed_1 = load_data('confirmed')
df_deaths_1 = load_data('deaths')
df_recovered_1 = load_data('recovered')

# clean the data
df_confirmed_new = df_confirmed_1[~df_confirmed_1['country'].str.contains(' \(total\)')].drop(['state'], axis=1, ).reset_index()
df_deaths_new = df_deaths_1[~df_deaths_1['country'].str.contains(' \(total\)')].drop(['state'], axis=1, ).reset_index()
df_recovered_new = df_recovered_1[~df_recovered_1['country'].str.contains(' \(total\)')].drop(['state'], axis=1, ).reset_index()


# get the data only for the latest date
Latest_Date = df_confirmed_new['date'].max()
df_confirmed_new = df_confirmed_new.loc[df_confirmed_new['date'] == Latest_Date]
df_deaths_new = df_deaths_new.loc[df_deaths_new['date'] == Latest_Date]
df_recovered_new = df_recovered_new.loc[df_recovered_new['date'] == Latest_Date]

# remove column 'date' as it is no more required
df_confirmed_new = df_confirmed_new.drop('date', axis=1)
df_deaths_new = df_deaths_new.drop('date', axis=1)
df_recovered_new = df_recovered_new.drop('date', axis=1)

# aggregate data
df_confirmed_new = (df_confirmed_new.sort_values(by=['country'])
                            .groupby(['country'])
                            .agg(sum)).reset_index()

df_deaths_new = (df_deaths_new.sort_values(by=['country'])
                    .groupby(['country'])
                    .agg(sum)).reset_index()

df_recovered_new = (df_recovered_new.sort_values(by=['country'])
                            .groupby(['country'])
                            .agg(sum)).reset_index()

# rename the column for 'cases' in respective datasets
df_confirmed_new = df_confirmed_new.rename(columns={"cases": "infections"})
df_deaths_new = df_deaths_new.rename(columns={"cases": "deaths"})
df_recovered_new = df_recovered_new.rename(columns={"cases": "recovered"})

# (inner)join the datasets for 'confirmed' and 'death' on 'country'
df_Master = pd.merge(df_confirmed_new, df_deaths_new, how='inner', on='country', left_on=None, right_on=None, left_index=False, right_index=False, sort=True)
df_Master = pd.merge(df_Master, df_recovered_new, how='inner', on='country', left_on=None, right_on=None, left_index=False, right_index=False, sort=True)

# rename the Countries
df_Master['country'] = df_Master['country'].replace({'Bosnia and Herzegovina':'Bosnia Herzegovina',
                                                     'Timor-Leste'           :'East Timor',
                                                     "Cote d'Ivoire"         :'Ivory Coast',
                                                     'Burma'                 :'Myanmar',
                                                     'Korea, South'          :'South Korea',
                                                     'Taiwan*'               :'Taiwan',
                                                     'US'                    :'United States of America',
                                                     'Holy See'              :'Vatican City'})

## remove the countries that have less than 200 confirmed cases
case_threshold = 200 
keep_countries = df_Master.loc[(df_Master['infections'] > case_threshold)].country
df_Master = df_Master.loc[df_Master['country'].isin(keep_countries)]

# calculate 'deaths' and 'recovery' per 1000 infections
df_Master['Deaths_per_1000'] = round(1000 * (df_Master['deaths']/df_Master['infections']), 2)
df_Master['Recovered_per_1000'] = round(1000 * (df_Master['recovered']/df_Master['infections']), 2)

# get countries with top 50 death rates
df_top_death = df_Master.sort_values(by=['Deaths_per_1000'], ascending=False)
df_top_death = df_top_death.head(50)

# get countries with top 50 recovery rates
df_top_recovered = df_Master.sort_values(by=['Recovered_per_1000'], ascending=False)
df_top_recovered = df_top_recovered.head(50)

# get rest of the countries
df_ROW = df_Master.loc[~df_Master['country'].isin(df_top_death.country)]
df_ROW = df_ROW.loc[~df_ROW['country'].isin(df_top_recovered.country)]

This graph shows the 50 countries that are facing highest rate of deaths per 1000 infections, across the world. As it is not so intuitive that the developed countries such as Belgium, France, Netherlands are among the top countries having highest death rate.

In [22]:
#collapse-hide
# make the bar-chart for countries on "deaths per 1000 infections"
alt.Chart(df_top_death).mark_bar(color='Orange').encode(
    x= alt.Y('country:N', sort='-y', title="Countries"),
    y=alt.Y('Deaths_per_1000', title="Deaths per 1000 infected people")
).properties(
    title='Countries with top death rates')

alt.Chart(...)

Further, we move on to the countries with best recovery rate. The graph below shows 50 countries with highest rate of recovery.

In [23]:
#collapse-hide
# make the bar-chart for countries on "recovery per 1000 infections"
alt.Chart(df_top_recovered).mark_bar(color='#00CC66').encode(
    x= alt.Y('country:N', sort='-y', title="Countries"),
    y=alt.Y('Recovered_per_1000', title="Recovered per 1000 infected people")
).properties(
    title='Countries with top recovery rates')

alt.Chart(...)

Finally we project the data from above graphs on to a scatter plot. The Orange and Green bubbles are from the above two bar charts. Additionally, the bubbles in Grey color are the rest of the countries.

In [24]:
#collapse-hide
# make the scatter plot for "Death Rates by Population Density" #00CC66
chart_high_recovery = alt.Chart(df_top_recovered).mark_circle(size=150, color='Red').encode(
    x=alt.Y('Recovered_per_1000',  title="Recovery per 1000 infections"),
    y=alt.Y('Deaths_per_1000',  title="Deaths per 1000 infections"),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

chart_high_death = alt.Chart(df_top_death).mark_circle(size=150, color='Orange').encode(
    x=alt.Y('Recovered_per_1000',  title="Recovery per 1000 infections"),
    y=alt.Y('Deaths_per_1000',  title="Deaths per 1000 infections"),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

chart_rest = alt.Chart(df_ROW).mark_circle(size=150, color='#C3C3C3').encode(
    x=alt.Y('Recovered_per_1000'),
    y=alt.Y('Deaths_per_1000'),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

(chart_high_recovery + chart_high_death + chart_rest).interactive()

alt.LayerChart(...)

## Appendix

Last Available Cases Per Million By Country:

In [25]:
#collapse-hide
label = 'Cases'
temp = pd.concat([x.copy() for x in data_countries_pc]).loc[lambda x: x.date >= '3/1/2020']

metric_name = f'{label} per Million'
temp.columns = ['Country', 'date', metric_name]
# temp.loc[:, 'month'] = temp.date.dt.strftime('%Y-%m')
temp.loc[:, f'Log of {label} per Million'] = temp[f'{label} per Million'].apply(lambda x: np.log(x))

temp.groupby('Country').last()

,date,Cases per Million,Log of Cases per Million
Country,,,
Brazil,2020-08-11,14608.892276,9.589386
China,2020-08-11,63.110960,4.144894
France,2020-08-11,3560.434324,8.177638
Germany,2020-08-11,2673.592716,7.891178
Iran,2020-08-11,4080.552265,8.313988
Italy,2020-08-11,4232.436274,8.350533
Japan,2020-08-11,394.573609,5.977806
Portugal,2020-08-11,5125.608176,8.542004
Romania,2020-08-11,3240.053277,8.083345
